In [12]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [13]:

df = pd.read_csv('srpski.csv', sep='\t')

df = df.drop(columns=['Rbr', 'SR', 'sr/sr', 'Naslov', 'Jezik'])
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.lower()


In [14]:

max_text_length = df['Tekst'].apply(lambda x: len(str(x).split())).max()

X_train, X_test, y_train, y_test = train_test_split(df['Tekst'], df['Autor'], test_size=0.2, random_state=42, stratify=df['Autor'])


In [15]:
early_stopping = EarlyStopping(
    patience=3,  
    monitor='accuracy',  
    restore_best_weights=True  
)


In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(sequences, maxlen=max_text_length)


In [17]:

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_text_length))
model.add(LSTM(100))
model.add(Dense(len(set(df['Autor']))+1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X, y_train, epochs=10, validation_split=0.2, callbacks=early_stopping)



Epoch 1/10
3/3 [==============================] - 8s 2s/step - loss: 2.7095 - accuracy: 0.0682 - val_loss: 2.7073 - val_accuracy: 0.0909
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 2.6969 - accuracy: 0.3295 - val_loss: 2.7075 - val_accuracy: 0.0909
Epoch 3/10
3/3 [==============================] - 7s 2s/step - loss: 2.6846 - accuracy: 0.4205 - val_loss: 2.7071 - val_accuracy: 0.0909
Epoch 4/10
3/3 [==============================] - 7s 2s/step - loss: 2.6679 - accuracy: 0.4545 - val_loss: 2.7079 - val_accuracy: 0.0909
Epoch 5/10
3/3 [==============================] - 7s 2s/step - loss: 2.6417 - accuracy: 0.3864 - val_loss: 2.7164 - val_accuracy: 0.1364
Epoch 6/10
3/3 [==============================] - 7s 2s/step - loss: 2.5944 - accuracy: 0.1818 - val_loss: 2.8821 - val_accuracy: 0.1364
Epoch 7/10
3/3 [==============================] - 7s 2s/step - loss: 2.5464 - accuracy: 0.1591 - val_loss: 2.7588 - val_accuracy: 0.0909


In [18]:

new_sequences = tokenizer.texts_to_sequences(X_test)
new_X = pad_sequences(new_sequences, maxlen=max_text_length)

predictions_train = model.predict(X)
predictions_test = model.predict(new_X)

evaluation_train = model.evaluate(X, y_train)
evaluation_test = model.evaluate(new_X, y_test)

1/1 [==============================] - 0s 470ms/step - loss: 2.6863 - accuracy: 0.1786


In [19]:
evaluation_train[1]

0.34545454382896423

In [20]:
evaluation_test[1]

0.1785714328289032

In [22]:
#model.save("rnn.keras")
